In [2]:
import requests
import pandas as pd
import numpy as np
import mysql.connector

# Loading GitHub Personal Access Token

In [3]:
with open("token.txt", "r") as file:
    GITHUB_TOKEN = file.read().strip() 

# GitHub API URL
GITHUB_API_URL = "https://api.github.com/search/repositories"

# Headers for authentication
authentication = {"Authorization": f"Token {GITHUB_TOKEN}"}

# List of 10 currently trending topics in the data world
topics = ["machine learning", "data visualization", "deep learning",
          "natural language processing", "artificial intelligence", "hypothesis testing",
          "time series analysis", "amazon sagemaker", "tensorflow", "Generative AI"]

# to fetch repositories for a specific topic
def fetch_repositories(topic, max_repos=100):
    repositories = []  # to store the retrieved repository data
    page = 1
    while len(repositories) < max_repos:
        parameter = {
            "q": topic,  # Query parameter for searching repositories by topic
            "sort": "stars",  # Sorting by stars to get the most popular repositories
            "order": "desc", # ordering in descending order
            "per_page": 100,  # Number of repositories per page
            "page": page  # Pagination control
        }
        response = requests.get(GITHUB_API_URL, headers=authentication, params=parameter)
        # If the API request fails 
        if response.status_code == 200:
            repos = response.json()["items"]
            repositories.extend(repos)
            if len(repos) <= 100:  # If fewer than 100 repos are returned, exit the loop
                break
        else:
            print(f"Failed to fetch data for {topic}: {response.status_code}")
            break
        page += 1

    return repositories

# List to store the extracted repository data
repositories_data = []

# Extract data for each topic
for topic in topics:
    print(f"Fetching repositories for topic: {topic}")
    repos = fetch_repositories(topic)
    for repo in repos:
        repositories_data.append({
            "Repository Name": repo["name"],
            "Owner": repo["owner"]["login"],
            "Description": repo["description"],
            "URL": repo["html_url"],
            "Programming Language": repo["language"],
            "Creation Date": repo["created_at"],
            "Last Updated Date": repo["updated_at"],
            "Number of Stars": repo["stargazers_count"],
            "Number of Forks": repo["forks_count"],
            "Number of Open Issues": repo["open_issues_count"],
            "License Type": repo["license"]["name"] if repo["license"] else "No License"
        })

# Convert the data to a DataFrame
df = pd.DataFrame(repositories_data)
df.head()

Fetching repositories for topic: machine learning
Fetching repositories for topic: data visualization
Fetching repositories for topic: deep learning
Fetching repositories for topic: natural language processing
Fetching repositories for topic: artificial intelligence
Fetching repositories for topic: hypothesis testing
Fetching repositories for topic: time series analysis
Fetching repositories for topic: amazon sagemaker
Fetching repositories for topic: tensorflow
Fetching repositories for topic: Generative AI


,Repository Name,Owner,Description,URL,Programming Language,Creation Date,Last Updated Date,Number of Stars,Number of Forks,Number of Open Issues,License Type
0,tensorflow,tensorflow,An Open Source Machine Learning Framework for ...,https://github.com/tensorflow/tensorflow,C++,2015-11-07T01:19:20Z,2024-10-09T03:29:49Z,185882,74237,5119,Apache License 2.0
1,transformers,huggingface,🤗 Transformers: State-of-the-art Machine Learn...,https://github.com/huggingface/transformers,Python,2018-10-29T13:56:00Z,2024-10-09T04:16:52Z,133092,26566,1420,Apache License 2.0
2,ML-For-Beginners,microsoft,"12 weeks, 26 lessons, 52 quizzes, classic Mach...",https://github.com/microsoft/ML-For-Beginners,HTML,2021-03-03T01:34:05Z,2024-10-09T04:58:56Z,69359,14354,9,MIT License
3,funNLP,fighting41love,中英文敏感词、语言检测、中外手机/电话归属地/运营商查询、名字推断性别、手机号抽取、身份证抽...,https://github.com/fighting41love/funNLP,Python,2018-08-21T11:20:39Z,2024-10-09T04:59:20Z,68129,14443,28,No License
4,awesome-machine-learning,josephmisiti,A curated list of awesome Machine Learning fra...,https://github.com/josephmisiti/awesome-machin...,Python,2014-07-15T19:11:19Z,2024-10-09T03:15:30Z,65636,14599,7,Other


In [4]:
df.shape

(1000, 11)

In [5]:
df.isnull().sum()

Repository Name            0
Owner                      0
Description               16
URL                        0
Programming Language     131
Creation Date              0
Last Updated Date          0
Number of Stars            0
Number of Forks            0
Number of Open Issues      0
License Type               0
dtype: int64

In [6]:
df["Description"].sample(3)

323    A curated list of speech and natural language ...
2      12 weeks, 26 lessons, 52 quizzes, classic Mach...
177    High-performance Toolkit for WebGL-based Data ...
Name: Description, dtype: object

Description Max length

In [7]:
max_description_length = df['Description'].str.len().max()
max_description_length

np.float64(80520.0)

In [54]:
df['Description'] = df['Description'].str[:255]

In [55]:
df.drop_duplicates(inplace=True)

In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 938 entries, 0 to 999
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Repository Name        938 non-null    object
 1   Owner                  938 non-null    object
 2   Description            922 non-null    object
 3   URL                    938 non-null    object
 4   Programming Language   817 non-null    object
 5   Creation Date          938 non-null    object
 6   Last Updated Date      938 non-null    object
 7   Number of Stars        938 non-null    int64 
 8   Number of Forks        938 non-null    int64 
 9   Number of Open Issues  938 non-null    int64 
 10  License Type           938 non-null    object
dtypes: int64(3), object(8)
memory usage: 87.9+ KB


In [57]:
df["Programming Language"].value_counts()

Programming Language
Python              298
Jupyter Notebook    207
TypeScript           57
JavaScript           39
C++                  33
Java                 28
HTML                 24
R                    21
Go                   18
Rust                  9
C#                    9
TeX                   8
Julia                 8
PHP                   5
Scala                 5
Shell                 5
C                     4
MATLAB                4
Ruby                  3
Common Lisp           2
Objective-C           2
CSS                   2
Swift                 2
Kotlin                2
Fortran               2
Matlab                2
Clojure               2
V                     1
Scheme                1
Vue                   1
Elixir                1
Emacs Lisp            1
Groff                 1
Hy                    1
HyPhy                 1
Dockerfile            1
Lean                  1
HCL                   1
Stata                 1
MDX                   1
Dart               

In [58]:
# Find NaN values in the 'Programming Language' column
total_nan_val = df['Programming Language'].isna().sum()
total_nan_val

np.int64(121)

In [59]:
fill_values = ['Python', 'Jupyter Notebook']

nan_val = df['Programming Language'].isna()

# Randomly select between 'Python' and 'Jupyter Notebook' to fill NaN values
df.loc[nan_val, 'Programming Language'] = np.random.choice(fill_values, size=nan_val.sum())

In [60]:
df.shape

(938, 11)

In [61]:
df.isna().sum()

Repository Name           0
Owner                     0
Description              16
URL                       0
Programming Language      0
Creation Date             0
Last Updated Date         0
Number of Stars           0
Number of Forks           0
Number of Open Issues     0
License Type              0
dtype: int64

In [62]:
df.dropna(inplace=True)

In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 922 entries, 0 to 999
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Repository Name        922 non-null    object
 1   Owner                  922 non-null    object
 2   Description            922 non-null    object
 3   URL                    922 non-null    object
 4   Programming Language   922 non-null    object
 5   Creation Date          922 non-null    object
 6   Last Updated Date      922 non-null    object
 7   Number of Stars        922 non-null    int64 
 8   Number of Forks        922 non-null    int64 
 9   Number of Open Issues  922 non-null    int64 
 10  License Type           922 non-null    object
dtypes: int64(3), object(8)
memory usage: 86.4+ KB


In [64]:
# Convert datetime columns from ISO format to MySQL compatible format
df['Creation Date'] = pd.to_datetime(df['Creation Date']).dt.strftime('%Y-%m-%d %H:%M:%S')
df['Last Updated Date'] = pd.to_datetime(df['Last Updated Date']).dt.strftime('%Y-%m-%d %H:%M:%S')

# Total Unique Values

In [65]:
unique_number = []
for i in df.columns:
    x = df[i].value_counts().count()
    unique_number.append(x)

pd.DataFrame(unique_number, index = df.columns, columns = ["Total Unique Values"])

,Total Unique Values
Repository Name,909
Owner,729
Description,921
URL,922
Programming Language,42
Creation Date,922
Last Updated Date,917
Number of Stars,777
Number of Forks,607
Number of Open Issues,248


In [66]:
df["License Type"].value_counts()

License Type
MIT License                                                   254
Apache License 2.0                                            214
No License                                                    189
Other                                                         105
MIT No Attribution                                             45
GNU General Public License v3.0                                33
BSD 3-Clause "New" or "Revised" License                        25
GNU Affero General Public License v3.0                         13
Creative Commons Zero v1.0 Universal                            9
Mozilla Public License 2.0                                      7
BSD 2-Clause "Simplified" License                               5
Creative Commons Attribution 4.0 International                  4
ISC License                                                     4
Creative Commons Attribution Share Alike 4.0 International      3
GNU Lesser General Public License v3.0                         

# Establish MySQL database connection

In [6]:
# Load environment variables from .env file
load_dotenv()

# MySQL connection using environment variables
connection = mysql.connector.connect(
    host=os.getenv("MYSQL_HOST"),           # AWS RDS endpoint
    user=os.getenv("MYSQL_USER"),           # MySQL username
    password=os.getenv("MYSQL_PASSWORD"),   # MySQL password
    database=os.getenv("MYSQL_DATABASE"),   # MySQL database name
    port=int(os.getenv("MYSQL_PORT", 3306)) # MySQL port (default 3306)
)


In [71]:
# Create a cursor object
cursor = connection.cursor()

# Create the repositories table if it doesn't exist
cursor.execute(
    """create table if not exists repositories(
    id INT AUTO_INCREMENT PRIMARY KEY,   -- Auto-incrementing ID
    repository_name VARCHAR(255),        -- Name of the repository
    owner VARCHAR(100),                  -- Repository owner's username
    description TEXT,                    -- Description of the repository
    url VARCHAR(255),                    -- URL to the repository
    programming_language VARCHAR(50),    -- Primary programming language
    creation_date DATETIME,              -- Creation date of the repository
    last_updated_date DATETIME,          -- Last updated date
    number_of_stars INT,                 -- Number of stars
    number_of_forks INT,                 -- Number of forks
    number_of_open_issues INT,           -- Number of open issues
    license_type VARCHAR(100)            -- License type (if any)
    )""");

# SQL query to insert data into the repositories table

In [72]:
insert_query = """
INSERT INTO repositories (
    repository_name, 
    owner, 
    description, 
    url, 
    programming_language, 
    creation_date, 
    last_updated_date, 
    number_of_stars, 
    number_of_forks, 
    number_of_open_issues, 
    license_type
) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
"""

# Insert each row in the DataFrame into MySQL
for index, row in df.iterrows():
    cursor.execute(insert_query, 
    (   row['Repository Name'], 
        row['Owner'], 
        row['Description'], 
        row['URL'],
        row['Programming Language'], 
        row['Creation Date'], 
        row['Last Updated Date'],
        row['Number of Stars'], 
        row['Number of Forks'], 
        row['Number of Open Issues'],
        row['License Type']
    ))

# Commit to save the changes
connection.commit()

# Close the cursor and connection
cursor.close()
connection.close()

print("Data successfully inserted into MySQL database")

Data was successfully inserted into MySQL database
